In [8]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.utils import upload_to_neuroglancer as upload
from brainlit.utils import upload_skeleton
import numpy as np
from pathlib import Path

In [9]:
import napari
%gui qt

In [10]:
data_dir = str(Path().resolve().parents[2] / "tests" / "data_octree")
dest_dir = str(Path() / "upload")

num_res = 2

# Uploading Brain Images in the Octree Format
## This is a script for uploading entire brain volumes, or uploading specific resolutions onto AWS or a local directory. 
## Data must be tif files arranged in folders where the highest level corresponds to a single, low res image
### Files should be arranged as octree with 1-8 indicating volume octant, Binary paths are used to stitch together images according to resolution

In [11]:
files, bin_paths, vox_size, tiff_dims = upload.get_volume_info(data_dir, num_res, channel = 0)
print("Low res files: " + str(files[0]))
print("\nHigh res files: " + str(files[1]))
print("---")
print("Single image binary: " + str(bin_paths[0]))
print("\nMultiple image binaries: " + str(bin_paths[1]))

got files and binary representations of paths.
got dimensions of volume
Low res files: ['/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/default.0.tif']

High res files: ['/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/7/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/6/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/1/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/8/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/4/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/3/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/2/default.0.tif', '/Users/bijanvarjavand/Documents/spring20/ndd/brainlit/tests/data_octree/5/default.0.tif']
---
Single image binary: [[]]

Multiple image binaries: [['110'], ['101'], ['

### Cloudvolume image layers are created with the number of resolutions in the original data

In [12]:
vols = upload.create_image_layer("file://" + dest_dir,tiff_dims, vox_size, num_res)
print("Number of volumes: " + str(len(vols)))
print("mips: " + str(vols[0].mip) + ' and ' + str(vols[1].mip))
print("Volumes info: " + str(vols[0].info))
print("---")
print("High res volume info: " + str(vols[0].info['scales'][0]))
print("\nLow res volume info: " + str(vols[1].info['scales'][1]))

Number of volumes: 2
mips: 1 and 0
Volumes info: {'data_type': 'uint16', 'num_channels': 1, 'scales': [{'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '9560_9741_31628', 'resolution': [9560, 9741, 31628], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}, {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '19120_19482_63256', 'resolution': [19120, 19482, 63256], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}], 'type': 'image'}
---
High res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '9560_9741_31628', 'resolution': [9560, 9741, 31628], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}

Low res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '19120_19482_63256', 'resolution': [19120, 19482, 63256], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}


### Uploading can be done with either Joblib parallel or non-parrallel sequential if the cpu power isn't there

In [13]:
%%capture
u1=upload.upload_chunks(vols[0], files[0], bin_paths[0], parallel=False) # Low res
u2=upload.upload_chunks(vols[1], files[1], bin_paths[1], parallel=False) # High res

# Visualize your data with NeuroglancerSession

In [14]:
mip = 1 # this can be either 0 or 1
tiff_dims = [528*(2-mip),400*(2-mip),208*(2-mip)]
ngl_sess = NeuroglancerSession(mip = mip, url = "file://" + dest_dir)
from cloudvolume import Bbox
img = ngl_sess.pull_bounds_img(Bbox((0,0,0), tiff_dims))










Downloading:   0%|          | 0/32 [00:00<?, ?it/s]








Downloading:   0%|          | 0/32 [00:00<?, ?it/s]








Downloading:   0%|          | 0/32 [00:00<?, ?it/s]








Downloading:   0%|          | 0/32 [00:00<?, ?it/s]








Downloading:  94%|█████████▍| 30/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 173.52it/s]








Downloading: 100%|██████████| 32/32 [00:00<00:00, 1

In [15]:
with napari.gui_qt():
    ngl_sess.napari_viewer(img)

In [9]:
# We can also load SWC files

In [ ]:
# 

In [ ]:
############# For now, I'm leaving the below as reference for building other notebooks. It will be deleted in the future.

In [ ]:
# 

In [16]:
# Uploading Neuron traces in .swc format locally
swc_dir = str(Path().resolve().parents[2] / "tests" / "data_swcs")
dest_dir_skel = str(Path() / "upload_segments")
dest_dir_s3 = "s3://mouse-light-viz/precomputed_volumes/brain1_lowres"
dest_dir_segments_s3 = "s3://mouse-light-viz/precomputed_volumes/brain1_lowres_segments"

In [17]:
data_dir = str(Path().resolve().parents[2] / "tests" / "data_octree")
(origin, vox_size, tiff_dims) = upload_skeleton.get_volume_info(data_dir, num_res)
vol = upload_skeleton.create_skeleton_layer(
    dest_dir_segments_s3, vox_size, tiff_dims, num_res
    # "file://"+dest_dir_skel, vox_size, tiff_dims, num_res
)
skeletons, segids = upload_skeleton.create_skel_segids(swc_dir, origin)
for skel in skeletons:
    vol.skeleton.upload(skel)

vol.skeleton.upload(skel)










converting swcs to neuroglancer format...: 100%|██████████| 1/1 [00:00<00:00, 37.46it/s]


In [14]:
ngl_sess = NeuroglancerSession(mip = 1, url = "file://" + dest_dir, url_segments="file://"+dest_dir_skel)

In [16]:
v_local = ngl_sess.cv_segments.skeleton.get(2)

In [41]:
img, bbox, vox = ngl_sess.pull_voxel(2, 300, 10, 10, 10)
skel = ngl_sess.cv_segments.skeleton.get(2)
vertex = skel.vertices[300]/ngl_sess.cv_segments.scales[1]["resolution"]
print(vertex)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  8.52it/s]
[237.83493724  76.73618597 100.73021532]


In [43]:
v_loc = vertex - bbox.to_list()[:3]

In [46]:
np.round(v_loc)

array([10., 10., 10.])

In [37]:
s3_path = "file://"+dest_dir_segments
seg_id = 2
df = read_s3(s3_path, seg_id, mip=themip)
df.head()

NameError: name 'dest_dir_segments' is not defined

In [39]:
bbox

Bbox([228, 67, 91],[249, 88, 112], dtype=int32)

In [ ]:
G = df_to_graph(df)

print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
print(G.nodes[1]['x'],G.nodes[1]['y'],G.nodes[1]['z'])

paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

In [47]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(np.squeeze(np.array(img)))
viewer.add_points(data=np.round(v_loc), size=1, face_color='green', name='vertex')

<Points layer 'vertex' at 0x18d35eef0>

In [13]:
# ngl_sess.pull_voxel(2, 6, nx=10, ny=10, nz=10) # currently mip mismatch and scale mismatch
from cloudvolume import Bbox


In [14]:
ngl_cloud = NeuroglancerSession(mip=1, url="s3://mouse-light-viz/precomputed_volumes/brain1_segments")

In [15]:
v_local = ngl_sess.cv.skeleton.get(2)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 458.90it/s]


In [16]:
v_cloud = ngl_cloud.cv.skeleton.get(2)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.93it/s]


In [17]:
v_local.vertices

array([[61594540. ,  6524150. , 21386580. ],
       [61596630. ,  6513191. , 21399428. ],
       [61596630. ,  6488230. , 21386580. ],
       ...,
       [61393936. ,  6774958.5, 21405630. ],
       [61385492. ,  6770693. , 21407436. ],
       [61381384. ,  6767023. , 21413338. ]], dtype=float32)

In [18]:
v_cloud.vertices

array([[3846942.8, 3798232.2, 4425521. ],
       [3849036.5, 3787273.2, 4438368.5],
       [3849036.5, 3762312.2, 4425521. ],
       ...,
       [3646341.2, 4049040.8, 4444572. ],
       [3637896. , 4044775.2, 4446376.5],
       [3633786.5, 4041105.2, 4452279. ]], dtype=float32)

In [23]:
v_local.vertices[100]/(2**4)

array([3862729.5 ,  364684.53, 1277418.9 ], dtype=float32)

In [24]:
v_cloud.vertices[100]

array([4056075.8, 3109035. , 3477641.8], dtype=float32)

In [19]:
def get_local_volume_around_vertex(ngl_sess, SEGID=2, VID=6, radius=10):
    skel = ngl_sess.cv.skeleton.get(SEGID)
    vertex = skel.vertices[VID]
    # below necessary to compensate for MIP differences
    # because the .swc file assumes 7 levels of resolution, 
    # but our test data only has the first 2
    scales = np.multiply(ngl_sess.cv.scales[1]["resolution"],2**3) # 3 for mip 1, 4 for mip 0
    voxel = np.round(np.divide(vertex, scales)).astype(int)
    bounds = Bbox(voxel, voxel)
    seed = bounds.to_list()
    shape = [radius, radius, radius]
    bounds = Bbox(np.subtract(seed[:3], shape), np.add(np.add(seed[3:], shape), 1))
    img = ngl_sess.cv.download(bounds)
    return img

In [20]:
# img = get_local_volume_around_vertex(ngl_sess, VID=50)
img = ngl_sess.pull_voxel(2, 50)
# 9891, 1102, 3449

Downloading: 100%|██████████| 1/1 [00:00<00:00, 319.06it/s]


OutOfBoundsError: Bbox([0, 0, 0],[1056, 800, 416], dtype=int32) did not fully contain the specified bounding box Bbox([4944, 550, 1724],[4947, 553, 1727], dtype=int32).

In [25]:
img2 = ngl_sess.pull_bounds_img(Bbox(
    (0,0,0), (int(9891/2**5), int(1102/2**5), int(3449/2**5))
    ))
img2 = ngl_sess.pull_bounds_img(Bbox(
    (0,0,0), (528, 400, 208)
))

Downloading:   0%|          | 0/32 [00:00<?, ?it/s]


In [26]:
with napari.gui_qt():
    ngl_sess.napari_viewer(img2)

remove TF  
allow stuff to run on Windows  
add more tests (actual unit tests!)

 - zoomed in retrieval of image
 - whole brain in napari

overlay SWC visualization (verify swc location, test!)  
(this is where it should be == this is where it is)

Why is the subvolume so bright?  
Janky with incomplete octree
 - swc file assumes 7 levels of resolution, we only have 2
 - interesting bug maybe? VID=100, radius=50
Use floats (possible cloudvolume PR)

=============

Clean up notebook
 - viz/vizualize.py and utils/swc.py
Plot a set number of points before and after (bfs on paths from origin point)

pull a bounding box around a single vertex
plot the other points within the box

still need to talk w Vikram about the info file generation, then once more confident we can to test uploads to s3 and load form s3?

want an s3 url brain1_lowres
run a notebook that does the above things

In [ ]:
# try anisotropic voxel size view